In [7]:
import json
from collections import defaultdict
import os
import pandas as pd
import docker
from tqdm import tqdm
from datasets import load_dataset
import argparse


In [8]:

def parse_args():
    """解析命令行参数"""
    parser = argparse.ArgumentParser(
        description="运行验证任务的配置参数",
        formatter_class=argparse.ArgumentDefaultsHelpFormatter  # 自动显示默认值
    )
    
    # 路径参数
    parser.add_argument(
        "--root_dir", "-r",
        type=str,
        default='/root/projects/main_projects/logs/run_validation/',
        help="日志存储根目录"
    )
    
    # 模型参数
    parser.add_argument(
        "--hf_name", "-m",
        type=str,
        default='zengliangcs/SWE-Fixer-Ours-2',
        help="Hugging Face仓库名称"
    )
    # run id
    parser.add_argument(
        "--run_id",
        type=str,
        default='0331',
        help="运行id号"
    )
    # 数据参数
    parser.add_argument(
        "--split", "-s",
        type=str,
        choices=['train', 'validation', 'test'],
        default='train',
        help="数据集划分类型"
    )
    
    return parser.parse_known_args()[0]

In [9]:
def get_validations(validation_logs, root_dir, test_type):
    total_instances_num = 0
    instance_with_report_num = 0
    root_dir = os.path.join(root_dir, test_type)
    for instance_id in os.listdir(root_dir):
        instance_id_dir = os.path.join(root_dir, instance_id)
        if os.path.isdir(instance_id_dir):
            total_instances_num += 1
            report_path = os.path.join(instance_id_dir, 'report.json')
            # 是否存在report.json
            if os.path.isfile(report_path):
                instance_with_report_num += 1
                try:
                    with open(report_path, 'r') as f:
                        data = json.load(f)
                    # 获取以子目录名为键的 resolved 值，默认为 False
                    # resolved = data.get(instance_id, {}).get('resolved', False)
                    tests_status =  data.get(instance_id, {}).get('tests_status', {})
                    if tests_status != {}:
                        pass_set = set(tests_status.get('PASS', []))
                        fail_set = set(tests_status.get('FAIL', []))
                        validation_logs[instance_id][f"{test_type}-PASS"] = pass_set
                        validation_logs[instance_id][f"{test_type}-FAIL"] = fail_set
                except (json.JSONDecodeError, IOError):
                    pass
    return total_instances_num, instance_with_report_num


In [10]:

def get_success_validation_data(root_dir):
    total_instances_num = {'gold': 0, 'empty': 0}
    instance_with_report_num = {'gold': 0, 'empty': 0}
    saved_num = 0
    validation_logs = defaultdict(lambda: defaultdict(set))
    success_validation_data = defaultdict(lambda: defaultdict(list))
    for test_type in ['gold', 'empty']:
        type_total_instances_num, type_instance_with_report_num = get_validations(validation_logs, root_dir, test_type)
        total_instances_num[test_type] = type_total_instances_num
        instance_with_report_num[test_type] = type_instance_with_report_num
    for instance_id, tests_status in validation_logs.items():
        fail_to_pass = tests_status["gold-PASS"] & tests_status["empty-FAIL"]
        pass_to_pass = tests_status["gold-PASS"] & tests_status["empty-PASS"]
        # 只保存fail_to_pass不为空的instance
        if len(fail_to_pass) > 0:
            saved_num += 1
            success_validation_data[instance_id]['FAIL_TO_PASS'] = list(fail_to_pass)
            success_validation_data[instance_id]['PASS_TO_PASS'] = list(pass_to_pass)
    return success_validation_data, total_instances_num, instance_with_report_num, saved_num


In [11]:

def filter_dataset(hf_name, split, success_validation_data):
    dataset = load_dataset(hf_name, split=split)
    def process_data(example):
        instance_id = example['instance_id']
        example['FAIL_TO_PASS'] = success_validation_data[instance_id]['FAIL_TO_PASS']
        example['PASS_TO_PASS'] = success_validation_data[instance_id]['PASS_TO_PASS']
        return example

    # 过滤数据并添加字段
    validated_instances = dataset.filter(
        lambda x: x['instance_id'] in success_validation_data.keys()
    ).map(process_data)

    return validated_instances


In [ ]:
args = parse_args()
print(f"当前配置：\n"
        f"日志目录：{args.root_dir}\n"
        f"运行ID：{args.run_id}\n"
        f"模型名称：{args.hf_name}\n"
        f"数据划分：{args.split}")

root_dir = os.path.join(args.root_dir, args.run_id)
success_validation_data, total_instances_num, instance_with_report_num, saved_num = \
    get_success_validation_data(root_dir)

print(f"instance总数：{total_instances_num=}")
print(f"生成report.json的instance总数：{instance_with_report_num=}")
print(f"保存的instance总数：{saved_num=}")

validated_instances = filter_dataset(args.hf_name, args.split, success_validation_data)
# 保存到本地
validated_instances.to_json(
    f"{args.hf_name}.jsonl", 
    orient="records", 
    lines=True,
    batch_size=10000,
    force_ascii=False
)


当前配置：
日志目录：/root/projects/main_projects/logs/run_validation/
模型名称：zengliangcs/SWE-Fixer-Ours-2
数据划分：train
instance总数：total_instances_num={'gold': 882, 'empty': 864}
生成report.json的instance总数：instance_with_report_num={'gold': 841, 'empty': 827}
保存的instance总数：saved_num=619


Map: 100%|██████████| 619/619 [00:00<00:00, 5042.71 examples/s]

Dataset({
    features: ['repo', 'pull_number', 'instance_id', 'issue_numbers', 'base_commit', 'patch', 'test_patch', 'problem_statement', 'hints_text', 'created_at', 'version', 'FAIL_TO_PASS', 'PASS_TO_PASS'],
    num_rows: 619
})


In [13]:
# Push validated instance images to Docker Hub
def push_validated_instances_images(
    instances, 
    namespace: str = "lycfight",
    instance_image_tag: str = "latest"
):
    client = docker.from_env()
    success = 0
    failed = 0
    
    print(f"Pushing {len(instances)} validated instance images to {namespace}...")
    with tqdm(total=len(instances)) as pbar:
        for instance in instances:
            instance_id = instance['instance_id']
            image_name = f"sweb.eval.x86_64.{instance_id.lower()}:{instance_image_tag}"
            new_image_name = f"{namespace}/{image_name}".replace("__", "_s_")
            
            try:
                image = client.images.get(image_name)
                image.tag(new_image_name)
                
                for line in client.images.push(new_image_name, stream=True, decode=True):
                    if 'error' in line:
                        raise Exception(line['error'])
                success += 1
                
            except Exception as e:
                print(f"Error pushing {image_name}: {e}")
                failed += 1
                
            pbar.update(1)
            pbar.set_postfix({
                "success": success,
                "failed": failed
            })
    
    print(f"Finished pushing images: {success} successful, {failed} failed")

In [ ]:
# Push images for validated instances
push_validated_instances_images(validated_instances)

Pushing 619 validated instance images to lycfight...


 14%|█▎        | 85/619 [00:00<00:00, 552.14it/s, success=0, failed=85]

Error pushing sweb.eval.x86_64.pypa__cibuildwheel-408:latest: 404 Client Error for http+docker://localhost/v1.48/images/sweb.eval.x86_64.pypa__cibuildwheel-408:latest/json: Not Found ("No such image: sweb.eval.x86_64.pypa__cibuildwheel-408:latest")
Error pushing sweb.eval.x86_64.pypa__cibuildwheel-455:latest: 404 Client Error for http+docker://localhost/v1.48/images/sweb.eval.x86_64.pypa__cibuildwheel-455:latest/json: Not Found ("No such image: sweb.eval.x86_64.pypa__cibuildwheel-455:latest")
Error pushing sweb.eval.x86_64.pypa__cibuildwheel-553:latest: 404 Client Error for http+docker://localhost/v1.48/images/sweb.eval.x86_64.pypa__cibuildwheel-553:latest/json: Not Found ("No such image: sweb.eval.x86_64.pypa__cibuildwheel-553:latest")
Error pushing sweb.eval.x86_64.pypa__cibuildwheel-829:latest: 404 Client Error for http+docker://localhost/v1.48/images/sweb.eval.x86_64.pypa__cibuildwheel-829:latest/json: Not Found ("No such image: sweb.eval.x86_64.pypa__cibuildwheel-829:latest")
Erro

 17%|█▋        | 106/619 [00:00<00:00, 552.14it/s, success=0, failed=106]

Error pushing sweb.eval.x86_64.networkx__networkx-5616:latest: 404 Client Error for http+docker://localhost/v1.48/images/sweb.eval.x86_64.networkx__networkx-5616:latest/json: Not Found ("No such image: sweb.eval.x86_64.networkx__networkx-5616:latest")
Error pushing sweb.eval.x86_64.networkx__networkx-5697:latest: 404 Client Error for http+docker://localhost/v1.48/images/sweb.eval.x86_64.networkx__networkx-5697:latest/json: Not Found ("No such image: sweb.eval.x86_64.networkx__networkx-5697:latest")
Error pushing sweb.eval.x86_64.networkx__networkx-5707:latest: 404 Client Error for http+docker://localhost/v1.48/images/sweb.eval.x86_64.networkx__networkx-5707:latest/json: Not Found ("No such image: sweb.eval.x86_64.networkx__networkx-5707:latest")
Error pushing sweb.eval.x86_64.networkx__networkx-5800:latest: 404 Client Error for http+docker://localhost/v1.48/images/sweb.eval.x86_64.networkx__networkx-5800:latest/json: Not Found ("No such image: sweb.eval.x86_64.networkx__networkx-5800:l

...

 61%|██████    | 378/619 [04:16<05:10,  1.29s/it, success=0, failed=378]

Error pushing sweb.eval.x86_64.pallets__click-2093:latest: denied: requested access to the resource is denied


 61%|██████    | 379/619 [04:17<05:12,  1.30s/it, success=0, failed=379]

Error pushing sweb.eval.x86_64.pallets__click-2094:latest: denied: requested access to the resource is denied


 61%|██████▏   | 380/619 [04:18<05:10,  1.30s/it, success=0, failed=380]